### 3.0 LLMs and Chat Models

In [1]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI()

In [3]:
a = llm.predict("How many planets are there?")
b = llm.predict("How many planets are there?")

In [4]:
a, b

('There are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, there are ongoing debates and explorations for the possible existence of a ninth planet in the outer solar system.',
 'There are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

### 3.1 Predict Messages

In [5]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

In [6]:
messages = [
    SystemMessage(content='you are a geograph expert. and you only reply in Korea.'),
    AIMessage(content='와우! 제 이름은 딩고에요!'),
    HumanMessage(content="맥시코와 태국 사이의 거리는 얼마인가. 그리고 너의 이름은 무엇이니?"),
]

chat.predict_messages(messages)

AIMessage(content='멕시코와 태국 사이의 거리는 약 14,000km 정도 되요. 제 이름은 딩고에요!')

### 3.2 Prompt Templates

In [7]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

**3.2.1 PromptTemplate**

In [8]:
template = PromptTemplate.from_template("what is the distance between {country_a} and {country_b}",)

template.format(country_a="Mexico", country_b="Thailand")

'what is the distance between Mexico and Thailand'

In [9]:
vars(template)

{'input_variables': ['country_a', 'country_b'],
 'input_types': {},
 'output_parser': None,
 'partial_variables': {},
 'template': 'what is the distance between {country_a} and {country_b}',
 'template_format': 'f-string',
 'validate_template': False}

In [10]:
prompt = template.format(country_a="Mexico", country_b="Thailand")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

**3.2.2 ChatPromptTemplate**

In [18]:
messages = ChatPromptTemplate.from_messages([
    ("system", "you are a geograph expert. and you only reply in {language}.",),
    ("ai", "와우! 제 이름은 {name}에요!",),
    ("human", "{country_a}와 {country_b} 사이의 거리는 얼마인가. 그리고 너의 이름은 무엇이니?",),],
)

vars(messages)

{'input_variables': ['country_a', 'country_b', 'language', 'name'],
 'input_types': {},
 'output_parser': None,
 'partial_variables': {},
 'messages': [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='you are a geograph expert. and you only reply in {language}.')),
  AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], template='와우! 제 이름은 {name}에요!')),
  HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country_a', 'country_b'], template='{country_a}와 {country_b} 사이의 거리는 얼마인가. 그리고 너의 이름은 무엇이니?'))],
 'validate_template': False}

In [21]:
prompt = messages.format_messages(
    language="Korea",
    name="딩고",
    country_a="Mexico",
    country_b="Thailand",
)

print(prompt)

[SystemMessage(content='you are a geograph expert. and you only reply in Korea.'), AIMessage(content='와우! 제 이름은 딩고에요!'), HumanMessage(content='Mexico와 Thailand 사이의 거리는 얼마인가. 그리고 너의 이름은 무엇이니?')]


In [20]:
chat.predict_messages(prompt)

AIMessage(content='멕시코와 태국 사이의 거리는 약 15,000km 정도 되요. 제 이름은 딩고에요!')

### 3.3 OutputParser and LCEL

**3.3.1 OutputParser**

In [27]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items =  text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()

p.parse("Hello, how, are, you?")

['Hello', 'how', 'are', 'you?']

In [35]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you ar asked will be answered with a comma seperated list of max {max_items} in lowercase. Do Not reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items = 10,
    question="What are the colors?"
)

print(prompt)

[SystemMessage(content='You are a list generating machine. Everything you ar asked will be answered with a comma seperated list of max 10 in lowercase. Do Not reply with anything else.'), HumanMessage(content='What are the colors?')]


In [36]:
chat.predict_messages(prompt)

AIMessage(content='red, blue, green, yellow, orange, purple, pink, black, white, brown')

In [39]:
result = chat.predict_messages(prompt)
p = CommaOutputParser()
parse_result = p.parse(result.content)
parse_result

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [40]:
type(parse_result)

list

**3.3.2 LCEL**

In [41]:
chain = template | chat | p

In [ ]:
vars(chain)

In [44]:
chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']